In [1]:
import os
import urllib.request
import zipfile
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import time
from datetime import datetime

In [2]:
from elasticsearch import Elasticsearch

In [3]:
es = Elasticsearch('elasticsearch-node:9200')

if es.indices.exists('stream-test'):
    es.indices.delete('stream-test')
    
    body={
        'mappings': {
            'properties': {
                'count': {'type': 'text'},
                'name': {'type': 'text'},
                'value': {'type': 'text'},
                'timestamp': {'type': 'text'}
            }
        }
    }
    
    es.indices.create(index='stream-test', body=body)
    
print(es.info())

{'name': '927d8d22d8c7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '038uq0CNScCWYvn2HwLEoA', 'version': {'number': '7.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8bec50e1e0ad29dad5653712cf3bb580cd1afcdf', 'build_date': '2020-01-15T12:11:52.313576Z', 'build_snapshot': False, 'lucene_version': '8.3.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars elasticsearch-hadoop-7.5.2/dist/elasticsearch-spark-20_2.11-7.5.2.jar pyspark-shell'
sc = SparkContext(appName="PythonSparkStreaming")
sc.setLogLevel("WARN")

In [5]:
ssc = StreamingContext(sc, 3)

In [6]:
print(os.getcwd())
json_folder_path = ('./sample/')

json_folder_path = ('./sample/')
json_files = [ x for x in os.listdir(json_folder_path ) if x.endswith("json") ]
json_merge = {}

for json_file in json_files:

    json_file_path = os.path.join(json_folder_path, json_file)

    with open (json_file_path) as f:

        data_dict = json.loads(f.read())  

        json_merge.update(data_dict)
        print(json_merge)


#parsed = stream.map(lambda x: format_data(x))
#stream = ssc.textFileStream(json_file_path) 

/home/jovyan
{'count': 5, 'name': 'Bilbo', 'value': 37, 'timestamp': 1581297174}
{'count': 3, 'name': 'Bilbo', 'value': 66, 'timestamp': 1581297172}
{'count': 4, 'name': 'Legolas', 'value': 52, 'timestamp': 1581297173}
{'count': 11, 'name': 'Frodo', 'value': 4, 'timestamp': 1581297180}
{'count': 7, 'name': 'Legolas', 'value': 56, 'timestamp': 1581297176}
{'count': 8, 'name': 'Aragorn', 'value': 78, 'timestamp': 1581297177}
{'count': 0, 'name': 'Legolas', 'value': 54, 'timestamp': 1581297169}
{'count': 6, 'name': 'Samwise', 'value': 54, 'timestamp': 1581297175}
{'count': 9, 'name': 'Frodo', 'value': 91, 'timestamp': 1581297178}
{'count': 2, 'name': 'Bilbo', 'value': 79, 'timestamp': 1581297171}
{'count': 12, 'name': 'Legolas', 'value': 57, 'timestamp': 1581297181}


In [7]:
es_write_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test',
    "es.input.json" : "yes",
    "es.mapping.id": "count"
}

In [8]:
rdd = sc.parallelize(json_merge)

print(json_merge)

{'count': 12, 'name': 'Legolas', 'value': 57, 'timestamp': 1581297181}


In [9]:
def format_data(x):
    return (json_merge)      

rdd = rdd.map(lambda x: format_data(x))

In [10]:
rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_write_conf
)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.saveAsNewAPIHadoopFile.
: org.apache.spark.SparkException: RDD element of type java.util.HashMap cannot be used
	at org.apache.spark.api.python.SerDeUtil$.pythonToPairRDD(SerDeUtil.scala:274)
	at org.apache.spark.api.python.PythonRDD$.saveAsNewAPIHadoopFile(PythonRDD.scala:577)
	at org.apache.spark.api.python.PythonRDD.saveAsNewAPIHadoopFile(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.com

In [ ]:
parsed.foreachRDD(lambda rdd: handler(rdd))

In [ ]:
parsed.pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.stop()

<b>Bulk Processing ES with Spark</b>

In [ ]:
sc = SparkContext(appName="PythonSparkReading")  
sc.setLogLevel("WARN")

In [ ]:
es_read_conf = {
    "es.nodes" : '927d8d22d8c7',
    "es.port" : '9200',
    "es.resource" : 'stream-test'
    }

In [ ]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_read_conf)

In [ ]:
es_rdd.take(5)

In [ ]:
es_rdd = es_rdd.map(lambda x: x[1])

In [ ]:
es_rdd.take(1)

In [ ]:
from pyspark.sql import SparkSession, SQLContext, Row

spark = SparkSession \
    .builder \
    .appName("Spark SQL") \
    .getOrCreate()

In [ ]:
df = es_rdd.map(lambda l: Row(**dict(l))).toDF()

In [ ]:
df.take(1)

In [ ]:
df \
    .groupby('name') \
    .count() \
    .collect()

In [ ]:
df \
    .filter(df.name == 'Legolas')\
    .take(1)

In [ ]:
sc.stop()